In [1]:
#pip install QuantLib-Python

**The objective of this notebook is to explore the QuantLib python library**
Most of the examples come from the documentation, but some examples also come from:
http://gouthamanbalaraman.com/blog/quantlib-python-tutorials-with-examples.html

In [19]:
import QuantLib as ql

Exloring the Time sub-module, which contains varios time-related classes. First, the date object.

In [3]:
#Date object 
date = ql.Date(31, 10, 2022) # 31 October, 2022
print(date)

day = date.dayOfMonth()
print(day)

month = date.month()
print(month)

year = date.year()
print(year)

print(date.weekday() == ql.Friday)

#Artithmetics with dates
tomorrow = date + 1  # add a day
print(tomorrow)

yesterday = date - 1  # subtract a day
print(yesterday)

next_month = date + ql.Period(1, ql.Months)
print(next_month)

next_week = date + ql.Period(1, ql.Weeks)
print(next_week)

next_year = date + ql.Period(1, ql.Years)
print(next_year)

#Logical operators
print(ql.Date(31, 10, 2022) > ql.Date(31, 10, 2021))

October 31st, 2022
31
10
2022
False
November 1st, 2022
October 30th, 2022
November 30th, 2022
November 7th, 2022
October 31st, 2023
True


Next, the Schedule object, which can be used to construct lists of dates (useful for coupon payments).

In [4]:
#Example 1
start = ql.Date(1, 1, 2019)
end = ql.Date(1, 1, 2020)
tenor = ql.Period(ql.Monthly)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)

schedule = ql.Schedule(start, end, tenor, calendar, ql.Following, ql.Following, ql.DateGeneration.Forward, False)

list(schedule)

[Date(2,1,2019),
 Date(1,2,2019),
 Date(1,3,2019),
 Date(1,4,2019),
 Date(1,5,2019),
 Date(3,6,2019),
 Date(1,7,2019),
 Date(1,8,2019),
 Date(3,9,2019),
 Date(1,10,2019),
 Date(1,11,2019),
 Date(2,12,2019),
 Date(2,1,2020)]

In [5]:
#Example 2
effectiveDate = ql.Date(15,6,2020)
terminationDate = ql.Date(15,6,2022)
frequency = ql.Period('6M')
calendar = ql.TARGET()
convention = ql.ModifiedFollowing
terminationDateConvention = ql.ModifiedFollowing
rule = ql.DateGeneration.Backward
endOfMonth = False
schedule = ql.Schedule(effectiveDate, terminationDate, frequency, calendar, convention, terminationDateConvention, rule, endOfMonth)

list(schedule)

[Date(15,6,2020),
 Date(15,12,2020),
 Date(15,6,2021),
 Date(15,12,2021),
 Date(15,6,2022)]

Next, looking into the DateGeneration class. The proper valuation of financial assets requires discounting future cash flows from the future, so generating the correct list of dates is crucial.

In [6]:
#Example 1
start = ql.Date(7,5,2020)
end = ql.Date(15,8,2020)

rules = {
    'Backward': ql.DateGeneration.Backward,
    'Forward': ql.DateGeneration.Forward,
    'Zero': ql.DateGeneration.Zero,
    'ThirdWednesDay': ql.DateGeneration.ThirdWednesday,
    'Twentieth': ql.DateGeneration.Twentieth,
    'TwentiethIMM': ql.DateGeneration.TwentiethIMM,
    'CDS': ql.DateGeneration.CDS

}

for name, rule in rules.items():
    schedule = ql.MakeSchedule(start, end, ql.Period('1m'), rule=rule)
    print(name, [dt for dt in schedule])

Backward [Date(7,5,2020), Date(15,5,2020), Date(15,6,2020), Date(15,7,2020), Date(15,8,2020)]
Forward [Date(7,5,2020), Date(7,6,2020), Date(7,7,2020), Date(7,8,2020), Date(15,8,2020)]
Zero [Date(7,5,2020), Date(15,8,2020)]
ThirdWednesDay [Date(7,5,2020), Date(17,6,2020), Date(15,7,2020), Date(15,8,2020)]
Twentieth [Date(7,5,2020), Date(20,5,2020), Date(20,6,2020), Date(20,7,2020), Date(20,8,2020)]
TwentiethIMM [Date(7,5,2020), Date(20,6,2020), Date(20,7,2020), Date(20,9,2020)]
CDS [Date(20,3,2020), Date(20,6,2020), Date(20,7,2020), Date(20,9,2020)]


Next, looking into the Calendar object. In particular, one can examine custom holidays.

In [7]:
#Example 1
cal = ql.TARGET()
mydate = ql.Date(1, ql.May, 2017)

print('Is BD :', cal.isBusinessDay(mydate))
print('Is Holiday :', cal.isHoliday(mydate))
print('Is Weekend :', cal.isWeekend(ql.Friday))
print('Is Last BD :', cal.isEndOfMonth(ql.Date(5, ql.April, 2018)))
print('Last BD :', cal.endOfMonth(mydate))

Is BD : False
Is Holiday : True
Is Weekend : False
Is Last BD : False
Last BD : May 31st, 2017


In [8]:
#Example 2
myCalendar = ql.WeekendsOnly()
days = [1,14,15,1,21,26,2,16,15,18,19,9,27,1,19,8,17,25,31]
months = [1,4,4,5,5,6,8,9,9,10,10,11,12,12,12,12]
name = ['Año Nuevo','Viernes Santo','Sabado Santo','Dia del Trabajo','Dia de las Glorias Navales','San Pedro y San Pablo','Elecciones Primarias','Dia de la Virgen del Carmen','Asuncion de la Virgen','Independencia Nacional','Glorias del Ejercito','Encuentro de dos mundos','Día de las Iglesias Evangélicas y Protestantes','Día de todos los Santos','Elecciones Presidenciales y Parlamentarias','Inmaculada Concepción','Segunda vuelta Presidenciales','Navidad','Feriado Bancario']
start_year = 2018
n_years = 10
for i in range(n_years+1):
    for x,y in zip(days,months):
        date = ql.Date(x,y,start_year+i)
        myCalendar.addHoliday(date)
        
holidaylist = ql.Calendar.holidayList(ql.TARGET(), ql.Date(1,1,2018), ql.Date(31,12,2023))
print(holidaylist)

(Date(1,1,2018), Date(30,3,2018), Date(2,4,2018), Date(1,5,2018), Date(25,12,2018), Date(26,12,2018), Date(1,1,2019), Date(19,4,2019), Date(22,4,2019), Date(1,5,2019), Date(25,12,2019), Date(26,12,2019), Date(1,1,2020), Date(10,4,2020), Date(13,4,2020), Date(1,5,2020), Date(25,12,2020), Date(1,1,2021), Date(2,4,2021), Date(5,4,2021), Date(15,4,2022), Date(18,4,2022), Date(26,12,2022), Date(7,4,2023), Date(10,4,2023), Date(1,5,2023), Date(25,12,2023), Date(26,12,2023))


Now, I evaluate the Interest Rate class

In [13]:
#Example 1
annualRate = 0.05
dayCount = ql.Actual360()
compoundType = ql.Compounded
frequency = ql.Annual
interestRate = ql.InterestRate(annualRate, dayCount, compoundType, frequency)

print(interestRate)

#The compoundFactor method will give how the investment will be worth after 't' years. If I invest 1 dollar:
investment1 = interestRate.compoundFactor(2.0)
print(investment1)

#The discountFactor method returns the reciprocal of the compoundFactor method.
discount1 = interestRate.discountFactor(2.0)
print(discount1)

#We can also do interest rate convertions using the equivalentRate method.
newFrequency = ql.Monthly
effectiveRate = interestRate.equivalentRate(compoundType, newFrequency, 1)
print(effectiveRate.rate())
#Here, I converted the original rate into a monthly compounding type. A 4.889% of monthly compounding is equivalent to a 5.0% of Anual compounding.

#To check the above, one can evaluate the discount factors
print(interestRate.discountFactor(1.0))
print(effectiveRate.discountFactor(1.0))
#This means that pricing bonds using either interest rate conversion should give the same net present value (barring some precision)


5.000000 % Actual/360 Annual compounding
1.1025
0.9070294784580498
0.04888948540378024
0.9523809523809523
0.9523809523809518


**Now, we model a fixed rate bond in QuantLib**
The bode is based on:
http://gouthamanbalaraman.com/blog/quantlib-bond-modeling.html

Let's consider a hypothetical bond with a par value of 100, that pays 6% coupon semi-annually issued on January 15th, 2015 and set to mature on January 15th, 2016. The bond will pay a coupon on July 15th, 2015 and January 15th, 2016. The par amount of 100 will also be paid on the January 15th, 2016.

To make things simpler, lets assume that we know the spot rates of the treasury as of January 15th, 2015. The annualized spot rates are 0.5% for 6 months and 0.7% for 1 year point. Lets calculate the fair value of this bond.

In [15]:
fairval = 3/pow(1+0.005, 0.5) + (100 + 3)/(1+0.007)
print(fairval)

105.27653992490681


Now, we perform the same task with QuantLib

In [17]:
#Creating the term structure and relevant variables
todaysDate = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = todaysDate
spotDates = [ql.Date(15, 1, 2015), ql.Date(15, 7, 2015), ql.Date(15, 1, 2016)]
spotRates = [0.0, 0.005, 0.007]
dayCount = ql.Thirty360(ql.Thirty360.BondBasis)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
interpolation = ql.Linear()
compounding = ql.Compounded
compoundingFrequency = ql.Annual
spotCurve = ql.ZeroCurve(spotDates, spotRates, dayCount, calendar, interpolation,
                             compounding, compoundingFrequency)
spotCurveHandle = ql.YieldTermStructureHandle(spotCurve)
print(spotCurveHandle)

<QuantLib.QuantLib.YieldTermStructureHandle; proxy of <Swig Object of type 'Handle< YieldTermStructure > *' at 0x000001D60248DCC0> >


In [25]:
#Creating the fixed rate bond
issueDate = ql.Date(15, 1, 2015)
maturityDate = ql.Date(15, 1, 2016)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
bussinessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule (issueDate, maturityDate, tenor, calendar, bussinessConvention,
                            bussinessConvention , dateGeneration, monthEnd)

list(schedule)

#Creating the coupon
dayCount = ql.Thirty360(ql.Thirty360.BondBasis)
couponRate = 0.06
coupons = [couponRate]

#Constructiny the FixedRateBond
settlementDays = 0
faceValue = 100
fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)

#Creating a BondEngine with the relevant term structure
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)

#Setting the bond to use with this Bond Engine
fixedRateBond.setPricingEngine(bondEngine)

#Obtaining the value of the bond today
value = fixedRateBond.NPV()
print(value)

print(fairval)

105.27653992490683
105.27653992490681


**Next, I perform a bootstrapping exercise for the yield curve using QuantLib**
The code is based on:
http://gouthamanbalaraman.com/blog/quantlib-term-structure-bootstrap-yield-curve.html

In [27]:
def print_curve(xlist, ylist, precision=3):
    """
    Method to print curve in a nice format
    """
    print ("----------------------")
    print ("Maturities\tCurve")
    print ("----------------------")
    for x,y in zip(xlist, ylist):
        print (x,"\t\t", round(y, precision))
    print ("----------------------")

The deposit rates and fixed rate bond rates are provided below. This example is based on Exhibit 5-5 given in Frank Fabozzi's Bond Markets, Analysis and Strategies, Sixth Edition.

In [28]:
# Deposit rates
depo_maturities = [ql.Period(6,ql.Months), ql.Period(12, ql.Months)]
depo_rates = [5.25, 5.5]

# Bond rates
bond_maturities = [ql.Period(6*i, ql.Months) for i in range(3,21)]
bond_rates = [5.75, 6.0, 6.25, 6.5, 6.75, 6.80, 7.00, 7.1, 7.15,
              7.2, 7.3, 7.35, 7.4, 7.5, 7.6, 7.6, 7.7, 7.8]

print_curve(depo_maturities+bond_maturities, depo_rates+bond_rates)

----------------------
Maturities	Curve
----------------------
6M 		 5.25
12M 		 5.5
18M 		 5.75
24M 		 6.0
30M 		 6.25
36M 		 6.5
42M 		 6.75
48M 		 6.8
54M 		 7.0
60M 		 7.1
66M 		 7.15
72M 		 7.2
78M 		 7.3
84M 		 7.35
90M 		 7.4
96M 		 7.5
102M 		 7.6
108M 		 7.6
114M 		 7.7
120M 		 7.8
----------------------


In [30]:
#Defining constants and conventions for object creating. It is assumed that some constants are the same for deposits and bonds.
calc_date = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = calc_date

calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
bussiness_convention = ql.Unadjusted
day_count = ql.Thirty360(ql.Thirty360.BondBasis)
end_of_month = True
settlement_days = 0
face_amount = 100
coupon_frequency = ql.Period(ql.Semiannual)
settlement_days = 0

The basic idea of bootstrapping using QuantLib is to use the deposit rates and bond rates to create individual helpers. Then use the combination of the two helpers to construct the yield curve.

In [34]:
#Creating depo rate 'helpers' from depo rates
depo_helpers = [ql.DepositRateHelper(ql.QuoteHandle(ql.SimpleQuote(r/100.0)),
                                     m,
                                     settlement_days,
                                     calendar,
                                     bussiness_convention,
                                     end_of_month,
                                     day_count )
                for r, m in zip(depo_rates, depo_maturities)]
print(depo_helpers)

[<QuantLib.QuantLib.DepositRateHelper; proxy of <Swig Object of type 'ext::shared_ptr< DepositRateHelper > *' at 0x000001D60248F6F0> >, <QuantLib.QuantLib.DepositRateHelper; proxy of <Swig Object of type 'ext::shared_ptr< DepositRateHelper > *' at 0x000001D60248F5D0> >]


In [36]:
#Creating fixed rate bond helpers from fixed rate bonds.
#IMPORTANT: It is assumed that the YTM given for the bonds are all par rates (bonds with coupons = YTM)
bond_helpers = []
for r, m in zip(bond_rates, bond_maturities):
    termination_date = calc_date + m
    schedule = ql.Schedule(calc_date,
                   termination_date,
                   coupon_frequency,
                   calendar,
                   bussiness_convention,
                   bussiness_convention,
                   ql.DateGeneration.Backward,
                   end_of_month)

    helper = ql.FixedRateBondHelper(ql.QuoteHandle(ql.SimpleQuote(face_amount)),
                                        settlement_days,
                                        face_amount,
                                        schedule,
                                        [r/100.0],
                                        day_count,
                                        bussiness_convention,
                                        )
    bond_helpers.append(helper)

print(bond_helpers)

[<QuantLib.QuantLib.FixedRateBondHelper; proxy of <Swig Object of type 'ext::shared_ptr< FixedRateBondHelper > *' at 0x000001D6025B0150> >, <QuantLib.QuantLib.FixedRateBondHelper; proxy of <Swig Object of type 'ext::shared_ptr< FixedRateBondHelper > *' at 0x000001D602175270> >, <QuantLib.QuantLib.FixedRateBondHelper; proxy of <Swig Object of type 'ext::shared_ptr< FixedRateBondHelper > *' at 0x000001D602175330> >, <QuantLib.QuantLib.FixedRateBondHelper; proxy of <Swig Object of type 'ext::shared_ptr< FixedRateBondHelper > *' at 0x000001D6025B04B0> >, <QuantLib.QuantLib.FixedRateBondHelper; proxy of <Swig Object of type 'ext::shared_ptr< FixedRateBondHelper > *' at 0x000001D602175060> >, <QuantLib.QuantLib.FixedRateBondHelper; proxy of <Swig Object of type 'ext::shared_ptr< FixedRateBondHelper > *' at 0x000001D602175240> >, <QuantLib.QuantLib.FixedRateBondHelper; proxy of <Swig Object of type 'ext::shared_ptr< FixedRateBondHelper > *' at 0x000001D60249A0C0> >, <QuantLib.QuantLib.FixedRa

In [38]:
#The yield curve is constructed putting both helpers together
rate_helpers = depo_helpers + bond_helpers
yieldcurve = ql.PiecewiseLogCubicDiscount(calc_date,
                                          rate_helpers,
                                         day_count)
print(yieldcurve)

<QuantLib.QuantLib.PiecewiseLogCubicDiscount; proxy of <Swig Object of type 'ext::shared_ptr< PiecewiseLogCubicDiscount > *' at 0x000001D60215F600> >


In [40]:
#Getting the spot rates
spots = []
tenors = []
for d in yieldcurve.dates():
    yrs = day_count.yearFraction(calc_date, d)
    compounding = ql.Compounded
    freq = ql.Semiannual
    zero_rate = yieldcurve.zeroRate(yrs, compounding, freq)
    tenors.append(yrs)
    eq_rate = zero_rate.equivalentRate(day_count,
                                       compounding,
                                       freq,
                                       calc_date,
                                       d).rate()
    spots.append(100*eq_rate)
    
print_curve(tenors, spots)

----------------------
Maturities	Curve
----------------------
0.0 		 0.0
0.5 		 5.25
1.0 		 5.426
1.5 		 5.761
2.0 		 6.02
2.5 		 6.283
3.0 		 6.55
3.5 		 6.822
4.0 		 6.87
4.5 		 7.095
5.0 		 7.205
5.5 		 7.257
6.0 		 7.31
6.5 		 7.429
7.0 		 7.485
7.5 		 7.543
8.0 		 7.671
8.5 		 7.802
9.0 		 7.791
9.5 		 7.929
10.0 		 8.072
----------------------
